In [1]:
import os
import random
import shutil
import time
import warnings
# os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
# Data loading code

##!!!! change the valdir to your data path
valdir = '/data/public/imagenet2012/val'   

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

batch_size = 256
val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ])),
    batch_size=batch_size, shuffle=False,
    num_workers=4, pin_memory=True)

In [11]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
           
            images = images.cuda()
            target = target.cuda()

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % 10 == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'
    
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].contiguous().view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res


## Arnold and iArnold

In [12]:
import copy

def arnold(img, key):
    N, a, b, c, d = key  #  key=[1,1,1,1,2]
    h, w = img.shape[: 2]
    new_img = copy.deepcopy(img)

    for i in range(N):
        
        for x in range(h):
            
            for y in range(w):
                
                nx = (a * x + b * y) % w
                ny = (c * x + d * y) % w 
                nx = int(nx)
                
                ny = int(ny)
                
                new_img[nx, ny] = img[x, y]
        img = copy.deepcopy(new_img)
    return new_img


def iarnold(img, key):
    N, a, b, c, d = key

    #matrix = np.mat([[a, b], [c, d]]).I

    [[a, b], [c, d]] = [[(a*b+1),-a],[-b,1]]
    return arnold(img, [N, a, b, c, d])

## Load the pre-trained models

In [13]:
import torchvision.models as models

cudnn.benchmark = True
criterion = nn.CrossEntropyLoss().cuda()
alexnet = models.alexnet(pretrained=True)
# resnet18 = models.resnet18(pretrained=True)

# vgg16 = models.vgg16(pretrained=True)
#densenet = models.densenet121(pretrained=True)
#inception = models.inception_v3(pretrained=True)
# googlenet = models.googlenet(pretrained=True)
#shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
#mobilenet = models.mobilenet_v2(pretrained=True)


# test the original accuracy
alexnet = alexnet.cuda()
validate(val_loader, alexnet, criterion)

Test: [  0/196]	Time  2.319 ( 2.319)	Loss 1.1817e+00 (1.1817e+00)	Acc@1  68.75 ( 68.75)	Acc@5  89.06 ( 89.06)
Test: [ 10/196]	Time  0.167 ( 1.091)	Loss 1.8619e+00 (1.5258e+00)	Acc@1  53.91 ( 64.49)	Acc@5  80.08 ( 84.23)
Test: [ 20/196]	Time  1.476 ( 1.297)	Loss 1.4823e+00 (1.4910e+00)	Acc@1  67.19 ( 65.23)	Acc@5  83.20 ( 84.77)
Test: [ 30/196]	Time  0.021 ( 1.475)	Loss 1.3381e+00 (1.4429e+00)	Acc@1  62.89 ( 66.15)	Acc@5  89.45 ( 85.37)
Test: [ 40/196]	Time  0.886 ( 1.473)	Loss 1.6524e+00 (1.5115e+00)	Acc@1  57.81 ( 63.71)	Acc@5  84.38 ( 84.90)
Test: [ 50/196]	Time  0.021 ( 1.559)	Loss 1.0528e+00 (1.4948e+00)	Acc@1  71.48 ( 63.26)	Acc@5  92.58 ( 85.52)
Test: [ 60/196]	Time  0.021 ( 1.531)	Loss 1.8547e+00 (1.4879e+00)	Acc@1  54.30 ( 63.28)	Acc@5  81.64 ( 85.79)
Test: [ 70/196]	Time  0.021 ( 1.616)	Loss 1.5965e+00 (1.4681e+00)	Acc@1  60.55 ( 63.75)	Acc@5  84.77 ( 86.03)
Test: [ 80/196]	Time  0.021 ( 1.597)	Loss 2.7413e+00 (1.4955e+00)	Acc@1  41.02 ( 63.38)	Acc@5  66.41 ( 85.62)
Test: [ 90

tensor(56.5200, device='cuda:0')

In [14]:
# see model structure
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [18]:
import torchvision.models as models
cudnn.benchmark = True
criterion = nn.CrossEntropyLoss().cuda()

# change this model
model = models.alexnet(pretrained=True)
# model = models.googlenet(pretrained=True)
#model = models.resnet101(pretrained=True)
#model = models.mobilenet_v2(pretrained=True)
#model = models.densenet121(pretrained=True)
#model = models.vgg16(pretrained=True)
model = model.cuda()

for name, param in model.named_parameters():
    print(name,'    ', param.size())
    
# AlexNet
# weight = model.features[0].weight
# weight = model.features[3].weight
# weight = model.features[6].weight
# weight = model.features[8].weight
# weight = model.features[10].weight
# weight = model.classifier[1].weight


# weight = model.inception4a.branch1.conv.weight
#weight = model.conv3.conv.weight 
# print('layer weights size is: ',weight.size())
# test original model acc
#validate(val_loader, model, criterion) 
# print(weight.shape)

features.0.weight      torch.Size([64, 3, 11, 11])
features.0.bias      torch.Size([64])
features.3.weight      torch.Size([192, 64, 5, 5])
features.3.bias      torch.Size([192])
features.6.weight      torch.Size([384, 192, 3, 3])
features.6.bias      torch.Size([384])
features.8.weight      torch.Size([256, 384, 3, 3])
features.8.bias      torch.Size([256])
features.10.weight      torch.Size([256, 256, 3, 3])
features.10.bias      torch.Size([256])
classifier.1.weight      torch.Size([4096, 9216])
classifier.1.bias      torch.Size([4096])
classifier.4.weight      torch.Size([4096, 4096])
classifier.4.bias      torch.Size([4096])
classifier.6.weight      torch.Size([1000, 4096])
classifier.6.bias      torch.Size([1000])


## Encryption

In [20]:
#Choose one layer (i.e., layer 3, namely features.3.weight)
weight = model.features[3].weight
weight_half = weight[:64,:64]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()

# secret key is [1,1,1,1,2]   \tau = 1  p=1, q=1
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])

end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec in python'.format(total))

model.features[3].weight.data[:64,:64] = torch.from_numpy(arn_weight).cuda()

validate(val_loader, model, criterion)  

layer half weights size is:  torch.Size([64, 64, 5, 5])
need 0.004524 sec
Test: [  0/196]	Time  3.280 ( 3.280)	Loss 4.0747e+00 (4.0747e+00)	Acc@1  24.61 ( 24.61)	Acc@5  45.70 ( 45.70)
Test: [ 10/196]	Time  0.021 ( 0.998)	Loss 4.0882e+00 (4.2193e+00)	Acc@1  18.75 ( 20.53)	Acc@5  46.48 ( 41.12)
Test: [ 20/196]	Time  1.567 ( 0.994)	Loss 5.1035e+00 (3.9342e+00)	Acc@1  14.45 ( 24.42)	Acc@5  27.73 ( 45.61)
Test: [ 30/196]	Time  0.021 ( 0.971)	Loss 4.2868e+00 (3.9508e+00)	Acc@1  15.23 ( 24.40)	Acc@5  38.67 ( 45.17)
Test: [ 40/196]	Time  0.358 ( 0.920)	Loss 4.7175e+00 (4.2960e+00)	Acc@1   9.38 ( 20.44)	Acc@5  30.08 ( 39.35)
Test: [ 50/196]	Time  0.021 ( 0.931)	Loss 3.4557e+00 (4.4173e+00)	Acc@1  25.39 ( 18.85)	Acc@5  47.27 ( 37.40)
Test: [ 60/196]	Time  0.021 ( 0.896)	Loss 3.7440e+00 (4.3694e+00)	Acc@1  18.36 ( 18.73)	Acc@5  45.70 ( 38.10)
Test: [ 70/196]	Time  0.021 ( 0.903)	Loss 4.5519e+00 (4.3016e+00)	Acc@1  19.53 ( 19.66)	Acc@5  36.72 ( 39.39)
Test: [ 80/196]	Time  0.021 ( 0.876)	Loss 4.77

tensor(21.1680, device='cuda:0')

## Decryption

In [22]:
start_i = time.time()

i_weight = iarnold(arn_weight, [1, 1,1,1,2])

end_i = time.time()
total_i = end_i - start_i

print('need {:.6f} sec in python'.format(total_i))
model.features[3].weight.data[:64,:64] = torch.from_numpy(i_weight).cuda()

validate(val_loader, model, criterion)  

need 0.007604 sec in python
Test: [  0/196]	Time  3.625 ( 3.625)	Loss 1.1817e+00 (1.1817e+00)	Acc@1  68.75 ( 68.75)	Acc@5  89.06 ( 89.06)
Test: [ 10/196]	Time  0.020 ( 0.939)	Loss 1.8619e+00 (1.5258e+00)	Acc@1  53.91 ( 64.49)	Acc@5  80.08 ( 84.23)
Test: [ 20/196]	Time  1.842 ( 0.922)	Loss 1.4823e+00 (1.4910e+00)	Acc@1  67.19 ( 65.23)	Acc@5  83.20 ( 84.77)
Test: [ 30/196]	Time  0.021 ( 0.883)	Loss 1.3381e+00 (1.4429e+00)	Acc@1  62.89 ( 66.15)	Acc@5  89.45 ( 85.37)
Test: [ 40/196]	Time  0.717 ( 0.848)	Loss 1.6524e+00 (1.5115e+00)	Acc@1  57.81 ( 63.71)	Acc@5  84.38 ( 84.90)
Test: [ 50/196]	Time  0.021 ( 0.862)	Loss 1.0528e+00 (1.4948e+00)	Acc@1  71.48 ( 63.26)	Acc@5  92.58 ( 85.52)
Test: [ 60/196]	Time  0.021 ( 0.839)	Loss 1.8547e+00 (1.4879e+00)	Acc@1  54.30 ( 63.28)	Acc@5  81.64 ( 85.79)
Test: [ 70/196]	Time  0.021 ( 0.861)	Loss 1.5965e+00 (1.4681e+00)	Acc@1  60.55 ( 63.75)	Acc@5  84.77 ( 86.03)
Test: [ 80/196]	Time  0.021 ( 0.842)	Loss 2.7413e+00 (1.4955e+00)	Acc@1  41.02 ( 63.38)	Acc@

tensor(56.5200, device='cuda:0')